In [1]:
# import the necessary packages
from collections import namedtuple
from skimage.filters import threshold_local
from skimage import segmentation
from skimage import measure
from imutils import perspective
import numpy as np
import imutils
import cv2

In [2]:
# to store information regarding the detected license plate.
LicensePlate = namedtuple("LicensePlateRegion", ["success", "plate", "thresh", "candidates"])

In [ ]:
def detectPlates():
    # this is the code from detectPlate.ipynb
    return

def detectCharacterCandidates(region):
    plate = perspective.four_point_transform(image, region)
    cv2.imshow("Perspective transform", imutils.resize(plate, width=400))
    V = cv2.split(cv2.cvtColor(plate, cv2.COLOR_BGR2HSV))[2] # extracting value channel from HSV
    T = threshold_local(V, 29, offset=15, method="gaussian") # extracting dark regions from light background or vv.
    thresh = (V > T).astype("uint8") * 255
    thresh = cv2.bitwise_not(thresh)

    # resize the license plate region to a canonical size
    plate = imutils.resize(plate, width=400)
    thresh = imutils.resize(thresh, width=400)
    cv2.imshow("Thresh", thresh)
    
    # CCA - connected component analysis
    labels = measure.label(thresh, neighbors=8, background=0)
    charCandidates = np.zeros(thresh.shape, dtype="uint8")
    
    for label in np.unique(labels):
        if label == 0: # background of license plate, hence ignore
            continue
            
        # By performing this masking, we are revealing only pixels that are part of the current connected component. 
        labelMask = np.zeros(thresh.shape, dtype="uint8")
        cnts = cv2.findContours(labelMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if imutils.is_cv2() else cnts[1]
        
        # ensure at least one contour was found in the mask
        if len(cnts) > 0:
            # grab the largest contour which corresponds to the component in the mask, then
            # grab the bounding box for the contour
            c = max(cnts, key=cv2.contourArea)
            (boxX, boxY, boxW, boxH) = cv2.boundingRect(c)

            # compute the aspect ratio, solidity, and height ratio for the component
            aspectRatio = boxW / float(boxH) # expected to be ideally taller rather than wide, at max a square
            solidity = cv2.contourArea(c) / float(boxW * boxH) # high value to avoid “noise”, such as dirt, bolts, etc
            heightRatio = boxH / float(plate.shape[0]) 
            # Large values of  heightRatio  indicate that the height of the (potential) character is similar 
            # to the license plate itself (and thus a likely character).

            # determine if the aspect ratio, solidity, and height of the contour pass
            # the rules tests
            keepAspectRatio = aspectRatio < 1.0
            keepSolidity = solidity > 0.15
            keepHeight = heightRatio > 0.4 and heightRatio < 0.95

            # check to see if the component passes all the tests
            if keepAspectRatio and keepSolidity and keepHeight:
                # compute the convex hull of the contour and draw it on the character
                # candidates mask
                hull = cv2.convexHull(c)
                cv2.drawContours(charCandidates, [hull], -1, 255, -1)
                
    # any components that are “touching” the borders of the image are removed
    charCandidates = segmentation.clear_border(charCandidates)
            
    return LicensePlate(success=True, plate=plate, thresh=thresh, candidates=charCandidates)

def detect():
    lpregions = detectPlates()
    
    for lpr in lpregions:
        lp = detectCharacterCandidates(lpr)
        if lp.success:
            yeild(lp,lpr)

In [4]:
from skimage.io import imread
car_image = imread("./vehicle.png", as_gray=True)
car_image.shape

(376, 603)